In [ ]:
# 0518 v1
# 새로운 방식 시도 - 좌표를 따라서 움직이는 게 아니라 마커 위치를 따라 이동
# a번 마커 위치에 도달 판단? 카메라 시야에 a번 마커가 중심에 오는지

#1. 아루코 설정 및 카메라 열기
#2. 순서대로 마커 추적
#3. 목표 마커가 카메라 중심 근처에 오면 다음 마커로 진행


In [ ]:
#!/usr/bin/env python3
import rospy
import cv2
import cv2.aruco as aruco
import numpy as np
from sensor_msgs.msg import Image
from geometry_msgs.msg import Twist
from cv_bridge import CvBridge

class ArucoFollowerROS:
    def __init__(self):
        rospy.init_node("aruco_follower_node")

        # 마커 추적 설정
        self.marker_length = 0.16
        self.target_ids = [12, 13, 14, 17, 16, 15]
        self.current_index = 0
        self.center_tolerance = 30  # 픽셀 기준

        # ROS
        self.bridge = CvBridge()
        self.cmd_pub = rospy.Publisher("/cmd_vel", Twist, queue_size=10)
        self.image_sub = rospy.Subscriber("/camera/rgb/image_raw", Image, self.image_callback)

        # 아루코
        self.aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
        self.detector_params = aruco.DetectorParameters()

        rospy.loginfo("✅ ArUco follower ROS 노드 시작!")

    def image_callback(self, msg):
        if self.current_index >= len(self.target_ids):
            rospy.loginfo("✅ 모든 마커를 추적 완료했습니다.")
            self.stop_robot()
            return

        # 이미지 변환
        frame = self.bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8")
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 마커 탐지
        corners, ids, _ = aruco.detectMarkers(gray, self.aruco_dict, parameters=self.detector_params)
        h, w = gray.shape
        center_x, center_y = w // 2, h // 2

        twist = Twist()
        found_target = False

        if ids is not None:
            ids = ids.flatten()
            target_id = self.target_ids[self.current_index]

            if target_id in ids:
                found_target = True
                idx = list(ids).index(target_id)
                corner = corners[idx][0]
                cx = int(corner[:, 0].mean())
                cy = int(corner[:, 1].mean())

                error_x = cx - center_x
                error_y = cy - center_y

                # 중심 도달 여부 판단
                if abs(error_x) < self.center_tolerance and abs(error_y) < self.center_tolerance:
                    rospy.loginfo(f"✅ 마커 {target_id} 중심 도달! 다음 마커로 이동.")
                    self.stop_robot()
                    rospy.sleep(1.0)
                    self.current_index += 1
                    return
                else:
                    # 중심 정렬 회전
                    twist.angular.z = -0.002 * error_x
                    twist.linear.x = 0.05  # 중심 근처에서도 천천히 전진

        # 마커 못찾은 경우 전진해서 탐색
        if not found_target:
            rospy.loginfo(f"❌ 마커 {self.target_ids[self.current_index]} 안 보임 → 전진 탐색")
            twist.linear.x = 0.1

        self.cmd_pub.publish(twist)

    def stop_robot(self):
        self.cmd_pub.publish(Twist())

if __name__ == "__main__":
    try:
        node = ArucoFollowerROS()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass
